In [87]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [88]:
empDF = spark.read.option("header",True).csv("/user/arnavmoutl12edu/module6/emp.csv")
deptDF = spark.read.option("header",True).csv("/user/arnavmoutl12edu/module6/dept.csv")

In [89]:
empDF.printSchema()

root
 |-- emp_id: string (nullable = true)
 |-- emp_name: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- emp_gender: string (nullable = true)
 |-- emp_salary: string (nullable = true)
 |-- emp_age: string (nullable = true)



In [90]:
deptDF.printSchema()

root
 |-- dept_id: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- dept_region: string (nullable = true)



In [92]:
empDF.show(5)

+------+--------+-------+----------+----------+-------+
|emp_id|emp_name|dept_id|emp_gender|emp_salary|emp_age|
+------+--------+-------+----------+----------+-------+
|  7001|     ABC|    100|         M|    107773|     30|
|  7002|     DEF|    101|         M|    122183|     25|
|  7003|     GHI|    102|         F|    147458|     26|
|  7004|     JKL|    103|         F|    140967|     29|
|  7005|     MNO|    104|         M|    136789|     28|
+------+--------+-------+----------+----------+-------+
only showing top 5 rows



In [93]:
deptDF.show(5)

+-------+---------+-----------+
|dept_id|dept_name|dept_region|
+-------+---------+-----------+
|    100|       HR|      India|
|    101|       IT|        USA|
|    102|  Finance|         UK|
|    103|    Audit|  Australia|
|    104|      CSR|     Canada|
+-------+---------+-----------+



In [98]:
empSelectDF = empDF.select('emp_id', 'emp_name')

In [99]:
empSelectDF.show(5)

+------+--------+
|emp_id|emp_name|
+------+--------+
|  7001|     ABC|
|  7002|     DEF|
|  7003|     GHI|
|  7004|     JKL|
|  7005|     MNO|
+------+--------+
only showing top 5 rows



In [94]:
joinDF = empDF.join(deptDF, empDF.dept_id == deptDF.dept_id)

In [100]:
joinDF.select('dept_id').show(5)

AnalysisException: "Reference 'dept_id' is ambiguous, could be: dept_id, dept_id.;"

In [101]:
joinDF = empDF.join(deptDF, empDF.dept_id == deptDF.dept_id).select(empDF['*'], deptDF['dept_name'], deptDF['dept_region'])

In [105]:
joinDF.printSchema()

root
 |-- emp_id: string (nullable = true)
 |-- emp_name: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- emp_gender: string (nullable = true)
 |-- emp_salary: string (nullable = true)
 |-- emp_age: integer (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- dept_region: string (nullable = true)



In [103]:
joinDF.write.csv("/user/arnavmoutl12edu/module6/employee_details.csv")

In [104]:
joinDF = joinDF.withColumn("emp_age", joinDF["emp_age"].cast('int'))

In [106]:
joinDF.groupby("dept_region").avg("emp_age").show()

+-----------+------------------+
|dept_region|      avg(emp_age)|
+-----------+------------------+
|      India|27.333333333333332|
|        USA|26.666666666666668|
|         UK|27.666666666666668|
|     Canada|27.333333333333332|
|  Australia|              28.0|
+-----------+------------------+



In [107]:
aggregateDF = joinDF.groupby("dept_name").agg({'emp_age':'max', 'emp_salary': 'min'})

In [108]:
aggregateDF.printSchema()

root
 |-- dept_name: string (nullable = true)
 |-- max(emp_age): integer (nullable = true)
 |-- min(emp_salary): string (nullable = true)



In [109]:
aggregateDF.show()

+---------+------------+---------------+
|dept_name|max(emp_age)|min(emp_salary)|
+---------+------------+---------------+
|      CSR|          29|         107928|
|    Audit|          30|         106135|
|       HR|          30|         104252|
|  Finance|          30|         127011|
|       IT|          30|         104298|
+---------+------------+---------------+



In [114]:
from pyspark.sql.functions import col as cols

In [115]:
aggregateDF.select('dept_name', cols('max(emp_age)').alias('max_age'), cols('min(emp_salary)').alias('min_salary')).show()

+---------+-------+----------+
|dept_name|max_age|min_salary|
+---------+-------+----------+
|      CSR|     29|    107928|
|    Audit|     30|    106135|
|       HR|     30|    104252|
|  Finance|     30|    127011|
|       IT|     30|    104298|
+---------+-------+----------+



In [116]:
aggregateDF.withColumnRenamed('max(emp_age)', 'max_age').withColumnRenamed('min(emp_salary)', 'min_salary').show()

+---------+-------+----------+
|dept_name|max_age|min_salary|
+---------+-------+----------+
|      CSR|     29|    107928|
|    Audit|     30|    106135|
|       HR|     30|    104252|
|  Finance|     30|    127011|
|       IT|     30|    104298|
+---------+-------+----------+



In [117]:
countDF = joinDF.groupby("dept_name").count()

In [118]:
countDF.show()

+---------+-----+
|dept_name|count|
+---------+-----+
|      CSR|    6|
|    Audit|    6|
|       HR|    6|
|  Finance|    6|
|       IT|    6|
+---------+-----+



In [ ]:
countDF.write.csv("/user/arnavmoutl12edu/module6/empcount.csv")

In [119]:
genderDF = joinDF.groupby("emp_gender").count()

In [120]:
genderDF.show()

+----------+-----+
|emp_gender|count|
+----------+-----+
|         F|   15|
|         M|   15|
+----------+-----+



In [ ]:
genderDF.printSchema()

In [121]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import sum as _sum

In [123]:
distDF = genderDF.withColumn('distribution', genderDF["count"]/_sum('count').over(Window.partitionBy())*100)

In [124]:
distDF.show()

+----------+-----+------------+
|emp_gender|count|distribution|
+----------+-----+------------+
|         F|   15|        50.0|
|         M|   15|        50.0|
+----------+-----+------------+



In [126]:
windowSpec = Window.partitionBy("dept_name").orderBy("emp_salary")
joinDF.withColumn('salary_rank', rank().over(windowSpec)).show()

+------+--------+-------+----------+----------+-------+---------+-----------+-----------+
|emp_id|emp_name|dept_id|emp_gender|emp_salary|emp_age|dept_name|dept_region|salary_rank|
+------+--------+-------+----------+----------+-------+---------+-----------+-----------+
|  7010|     BCD|    104|         M|    107928|     29|      CSR|     Canada|          1|
|  7020|     FGH|    104|         F|    109065|     29|      CSR|     Canada|          2|
|  7030|      GH|    104|         F|    128123|     25|      CSR|     Canada|          3|
|  7015|     QRS|    104|         M|    132422|     25|      CSR|     Canada|          4|
|  7025|     UVW|    104|         M|    134155|     28|      CSR|     Canada|          5|
|  7005|     MNO|    104|         M|    136789|     28|      CSR|     Canada|          6|
|  7009|     YZA|    103|         M|    106135|     26|    Audit|  Australia|          1|
|  7014|     NOP|    103|         F|    109075|     26|    Audit|  Australia|          2|
|  7029|  

# Reading from RDD

In [127]:
sc = spark.sparkContext

In [130]:
rdd1 = sc.textFile("/user/arnavmoutl12edu/module6/sales_large.csv")

In [129]:
rdd1 = rdd1.toDF().limit(10).rdd

TypeError: Can not infer schema for type: <class 'str'>

In [131]:
rdd2 = rdd1.map(lambda x: x.split(',')[:6])

In [132]:
rdd2.take(1)

[['Australia and Oceania',
  'Palau',
  'Office Supplies',
  'Online',
  'H',
  '3/6/2016']]

In [133]:
salesDF = spark.createDataFrame(rdd2)

In [134]:
salesDF.show(5)

+--------------------+-------+---------------+-------+---+---------+
|                  _1|     _2|             _3|     _4| _5|       _6|
+--------------------+-------+---------------+-------+---+---------+
|Australia and Oce...|  Palau|Office Supplies| Online|  H| 3/6/2016|
|              Europe| Poland|      Beverages| Online|  L|4/18/2010|
|       North America| Canada|         Cereal| Online|  M| 1/8/2015|
|              Europe|Belarus|         Snacks| Online|  C|1/19/2014|
|Middle East and N...|   Oman|         Cereal|Offline|  H|4/26/2019|
+--------------------+-------+---------------+-------+---+---------+
only showing top 5 rows



In [135]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType

In [136]:
schema = StructType([StructField("region",StringType(),True), \
    StructField("country",StringType(),True), \
    StructField("item_type",StringType(),True), \
    StructField("sales_cannel", StringType(), True), \
    StructField("order_priority", StringType(), True), \
    StructField("order_date", StringType(), True) \
  ])

In [137]:
salesDF = spark.createDataFrame(data = rdd2.take(5), schema = schema)

In [138]:
salesDF.printSchema()

root
 |-- region: string (nullable = true)
 |-- country: string (nullable = true)
 |-- item_type: string (nullable = true)
 |-- sales_cannel: string (nullable = true)
 |-- order_priority: string (nullable = true)
 |-- order_date: string (nullable = true)



In [139]:
salesDF.show(5)

+--------------------+-------+---------------+------------+--------------+----------+
|              region|country|      item_type|sales_cannel|order_priority|order_date|
+--------------------+-------+---------------+------------+--------------+----------+
|Australia and Oce...|  Palau|Office Supplies|      Online|             H|  3/6/2016|
|              Europe| Poland|      Beverages|      Online|             L| 4/18/2010|
|       North America| Canada|         Cereal|      Online|             M|  1/8/2015|
|              Europe|Belarus|         Snacks|      Online|             C| 1/19/2014|
|Middle East and N...|   Oman|         Cereal|     Offline|             H| 4/26/2019|
+--------------------+-------+---------------+------------+--------------+----------+



In [ ]:
salesDF.createOrReplaceView()

In [ ]:
spark.stop()